In [11]:
import requests
import pandas as pd
import math
import time

def 下載股價資訊(年份,股票代碼):
    月份列表 = [f"{str(年份)}{str(m).zfill(2)}01" for m in range(1, 13)]
    all_data = []
    for date_str in 月份列表:
        url = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={股票代碼}"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        data = response.json()

        if data["stat"] != "OK":
            continue

        df = pd.DataFrame(data["data"], columns=data["fields"])

        # 處理收盤價
        df["收盤價"] = pd.to_numeric(df["收盤價"].str.replace(",", ""), errors="coerce")

        all_data.append(df[["日期", "收盤價"]])
        time.sleep(1)  # 避免請求過快

    return pd.concat(all_data).dropna().sort_values("日期").reset_index(drop=True)

# 下載資料
股票代碼 = "2330"
df = 下載股價資訊(月份,股票代碼)

r_mean=df["收盤價"].pct_change().dropna().mean()
f"{r_mean:.2%}"

In [24]:
"""
HTTP呼叫模式
參考Message API資料，以request呼叫
https://developers.line.biz/en/reference/messaging-api/#send-reply-message
"""
import requests
import pandas as pd
import math
import time
import requests
from flask import Flask, request
def 下載股價資訊(年份,股票代碼):
    月份列表 = [f"{年份}{str(m).zfill(2)}01" for m in range(1, 13)]
    all_data = []
    for date_str in 月份列表:
        url = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={股票代碼}"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        data = response.json()

        if data["stat"] != "OK":
            continue

        df = pd.DataFrame(data["data"], columns=data["fields"])

        # 處理收盤價
        df["收盤價"] = pd.to_numeric(df["收盤價"].str.replace(",", ""), errors="coerce")

        all_data.append(df[["日期", "收盤價"]])
        time.sleep(1)  # 避免請求過快

    return pd.concat(all_data).dropna().sort_values("日期").reset_index(drop=True)
# 初始化 Flask 應用程式
app = Flask(__name__)

# 設定 LINE BOT Token
BOT_TOKEN = "貼上TOKEN"

# 傳送文字訊息函數
def send_text_message(reply_token, text):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + BOT_TOKEN
    }
    
    payload = {
        "replyToken": reply_token,
        "messages": [text]
    }
    
    # 發送 POST 請求至 LINE Messaging API
    response = requests.post(
        "https://api.line.me/v2/bot/message/reply",
        headers=headers,
        json=payload
    )
    return response

# LINE Webhook 入口
@app.route("/", methods=['POST'])
def linebot():
    global 年份
    # 取得使用者傳來的資料
    data = request.get_json()
    print(data)
    
    # 提取 replyToken
    reply_token = data['events'][0]['replyToken']
    if data['events'][0]['message']['text'][0:2] == "年份":
        年份=data['events'][0]['message']['text'][2:6]
        response = send_text_message(reply_token, {
    "type": "text",
    "text": "請再輸入股票代碼XXXX"
})
    elif data['events'][0]['message']['text'][0:4] == "股票代碼":
        股票代碼=data['events'][0]['message']['text'][4:]
        df = 下載股價資訊(年份,股票代碼)
        r_mean=df["收盤價"].pct_change().dropna().mean()        
        stock_data={
    "type": "text",
    "text": f"{r_mean:.2%}"
}
        response = send_text_message(reply_token, stock_data)
    else:
        print("別鬧")
        response = send_text_message(reply_token, {
    "type": "text",
    "text": "無法辨識，請輸入年份XXXX或股票代碼XXXX"
})
    
    if response.status_code == 200:
        return "OK", 200
    else:
        print("發送訊息失敗:", response.status_code, response.text)
        return "Error", 400

# 啟動 Flask 伺服器
if __name__ == "__main__":
    app.run(port=3000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3000
Press CTRL+C to quit


{'destination': 'U6ca4028e7d889a04f331e03f0252bf24', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '570706606527611188', 'quoteToken': 'Vn1qrDPz7BtisEJUdbcb4psOvRQnNrPwS71p-VDgiCpm8AjYwujjXyfG0nQ6Algai47QzM2QbcqDxhrRVFuMlTGAXiEVSL3uyR2sLzqcCZHaVKIi5xRQDxWngK5ucBKxUxRiAzy0ZkgHDqNKaNtlFw', 'text': '年份2023'}, 'webhookEventId': '01K0KE2D5KE09E8KN20XF26PM1', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1752998949560, 'source': {'type': 'user', 'userId': 'U0d28e97fd476a5689543684635550db4'}, 'replyToken': '7c4b3ed29bb34f889d43096b7d3a01c0', 'mode': 'active'}]}


127.0.0.1 - - [20/Jul/2025 16:09:10] "POST / HTTP/1.1" 200 -


{'destination': 'U6ca4028e7d889a04f331e03f0252bf24', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '570706620032746051', 'quoteToken': 'l5ytwbOUyTXM5HfzbS506ZoNImLVE155AY-6taSmlX70xd6RZhsE548kVwWI9E4zmxYi5brK82LDg06IiAFNpz2fUwmPYWdogvPuu9ozC9GDazFCogdPA8jTDkJcykQhgKZClBQhoL5HlLJb5xDBZg', 'text': '股票代碼2330'}, 'webhookEventId': '01K0KE2MXWZ49G69XSZ0TKC8TD', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1752998957598, 'source': {'type': 'user', 'userId': 'U0d28e97fd476a5689543684635550db4'}, 'replyToken': 'a635ede210544fec919f3975218421cc', 'mode': 'active'}]}


127.0.0.1 - - [20/Jul/2025 16:09:31] "POST / HTTP/1.1" 200 -
